In [9]:
!pip3 install webdriver-manager
!pip3 install beautifulsoup4

!pip3 install schedule

In [3]:
from webdriver_manager.chrome import ChromeDriverManager

old_path = ChromeDriverManager().install()
!mv "{old_path}" .
!ls | grep chromedriver


chromedriver


In [7]:

import time
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

# Setup WebDriver
def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Run in background
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

# Function to scrape Indeed
def scrape_indeed():
    driver = setup_driver()
    driver.get("https://www.indeed.com/jobs?q=Data+Science&l=United+States")

    time.sleep(5)  # Wait for page to load

    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()  # Close driver after fetching page source

    jobs = []
    for job_card in soup.find_all("div", class_="job_seen_beacon"):
        title = job_card.find("h2").text.strip() if job_card.find("h2") else "N/A"
        company = job_card.find("span", class_="companyName").text.strip() if job_card.find("span", class_="companyName") else "N/A"
        location = job_card.find("div", class_="companyLocation").text.strip() if job_card.find("div", class_="companyLocation") else "N/A"
        link = "https://www.indeed.com" + job_card.find("a")["href"] if job_card.find("a") else "N/A"

        jobs.append({
            "Source": "Indeed",
            "Title": title,
            "Company": company,
            "Location": location,
            "URL": link,
            "Date Scraped": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        })

    return jobs

# Function to scrape LinkedIn (dynamic approach)
def scrape_linkedin():
    driver = setup_driver()
    driver.get("https://www.linkedin.com/jobs/search/?keywords=Data%20Science&location=United%20States")

    time.sleep(5)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    jobs = []
    for job_card in soup.find_all("div", class_="base-card"):
        title = job_card.find("h3", class_="base-search-card__title").text.strip() if job_card.find("h3", class_="base-search-card__title") else "N/A"
        company = job_card.find("h4", class_="base-search-card__subtitle").text.strip() if job_card.find("h4", class_="base-search-card__subtitle") else "N/A"
        location = job_card.find("span", class_="job-search-card__location").text.strip() if job_card.find("span", class_="job-search-card__location") else "N/A"
        link = job_card.find("a")["href"] if job_card.find("a") else "N/A"

        jobs.append({
            "Source": "LinkedIn",
            "Title": title,
            "Company": company,
            "Location": location,
            "URL": link,
            "Date Scraped": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        })

    return jobs

# Function to save jobs to CSV
def save_to_csv(jobs, filename="job_listings.csv"):
    df = pd.DataFrame(jobs)
    df.to_csv(filename, mode='a', index=False, header=not pd.io.common.file_exists(filename))

# Main automation loop
def main():
    while True:
        print("Scraping new jobs...")

        jobs = scrape_indeed() + scrape_linkedin()
        save_to_csv(jobs)

        print(f"Scraping completed at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}. Data saved.")

        #time.sleep(1800)  # Run every 30 minutes

if __name__ == "__main__":
    main()


Scraping new jobs...
Scraping completed at 2025-03-17 17:53:21. Data saved.
Scraping new jobs...
Scraping completed at 2025-03-17 17:53:36. Data saved.
Scraping new jobs...
Scraping completed at 2025-03-17 17:53:50. Data saved.
Scraping new jobs...
Scraping completed at 2025-03-17 17:54:04. Data saved.
Scraping new jobs...
Scraping completed at 2025-03-17 17:54:19. Data saved.
Scraping new jobs...
Scraping completed at 2025-03-17 17:54:33. Data saved.
Scraping new jobs...
Scraping completed at 2025-03-17 17:54:48. Data saved.
Scraping new jobs...
Scraping completed at 2025-03-17 17:55:02. Data saved.
Scraping new jobs...
Scraping completed at 2025-03-17 17:55:17. Data saved.
Scraping new jobs...
Scraping completed at 2025-03-17 17:55:31. Data saved.
Scraping new jobs...
Scraping completed at 2025-03-17 17:55:46. Data saved.
Scraping new jobs...
Scraping completed at 2025-03-17 17:56:00. Data saved.
Scraping new jobs...
Scraping completed at 2025-03-17 17:56:14. Data saved.
Scraping new

KeyboardInterrupt: 

In [1]:
import time
import pandas as pd
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

# Setup WebDriver
def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Run in background
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

# Function to scrape Indeed
def scrape_indeed():
    driver = setup_driver()
    driver.get("https://www.indeed.com/jobs?q=Data+Science&l=United+States")

    time.sleep(5)  # Allow time for page load

    # Scroll to load more jobs
    for _ in range(3):
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
        time.sleep(2)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    jobs = []
    cutoff_time = datetime.now() - timedelta(days=1)  # 24-hour filter

    for job_card in soup.find_all("div", class_="job_seen_beacon"):
        title = job_card.find("h2").text.strip() if job_card.find("h2") else "N/A"
        company = job_card.find("span", class_="companyName").text.strip() if job_card.find("span", class_="companyName") else "N/A"
        location = job_card.find("div", class_="companyLocation").text.strip() if job_card.find("div", class_="companyLocation") else "N/A"
        link = "https://www.indeed.com" + job_card.find("a")["href"] if job_card.find("a") else "N/A"
        posted_text = job_card.find("span", class_="date").text.strip() if job_card.find("span", class_="date") else "N/A"

        # Convert posted time to datetime
        if "hour" in posted_text or "just" in posted_text or "minutes" in posted_text:
            posted_date = datetime.now()
        elif "day" in posted_text:
            days_ago = int(posted_text.split()[0])
            posted_date = datetime.now() - timedelta(days=days_ago)
        else:
            posted_date = datetime.now() - timedelta(days=2)  # Assume older if unknown

        if posted_date >= cutoff_time:
            jobs.append({
                "Source": "Indeed",
                "Title": title,
                "Company": company,
                "Location": location,
                "URL": link,
                "Posted Date": posted_date.strftime("%Y-%m-%d"),
                "Date Scraped": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            })

    return jobs

# Function to scrape LinkedIn dynamically
def scrape_linkedin():
    driver = setup_driver()
    driver.get("https://www.linkedin.com/jobs/search/?keywords=Data%20Science&location=United%20States")

    time.sleep(5)

    # Scroll down to load more jobs
    for _ in range(3):
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
        time.sleep(2)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    jobs = []
    cutoff_time = datetime.now() - timedelta(days=1)  # 24-hour filter

    for job_card in soup.find_all("div", class_="base-card"):
        title = job_card.find("h3", class_="base-search-card__title").text.strip() if job_card.find("h3", class_="base-search-card__title") else "N/A"
        company = job_card.find("h4", class_="base-search-card__subtitle").text.strip() if job_card.find("h4", class_="base-search-card__subtitle") else "N/A"
        location = job_card.find("span", class_="job-search-card__location").text.strip() if job_card.find("span", class_="job-search-card__location") else "N/A"
        link = job_card.find("a")["href"] if job_card.find("a") else "N/A"
        posted_text = job_card.find("time")["datetime"] if job_card.find("time") else "N/A"

        # Convert posted time to datetime
        if posted_text != "N/A":
            posted_date = datetime.strptime(posted_text, "%Y-%m-%d")
        else:
            posted_date = datetime.now() - timedelta(days=2)  # Assume older if unknown

        if posted_date >= cutoff_time:
            jobs.append({
                "Source": "LinkedIn",
                "Title": title,
                "Company": company,
                "Location": location,
                "URL": link,
                "Posted Date": posted_date.strftime("%Y-%m-%d"),
                "Date Scraped": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            })

    return jobs

# Function to save jobs to CSV (New jobs at top)
def save_to_csv(jobs, filename="job_listings.csv"):
    df_new = pd.DataFrame(jobs)

    try:
        df_existing = pd.read_csv(filename)
        df_combined = pd.concat([df_new, df_existing], ignore_index=True)  # New jobs at top
    except FileNotFoundError:
        df_combined = df_new

    df_combined.to_csv(filename, index=False)

# Main function with automation condition
def main(automation=True):
    while True:
        print("Scraping new jobs...")

        jobs = scrape_indeed() + scrape_linkedin()
        if jobs:
            save_to_csv(jobs)
            print(f"Scraping completed at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}. Data saved.")
        else:
            print("No new jobs found.")

        if not automation:
            break  # Run once and exit

        time.sleep(43200)  # Run every 12 hours

# Run script
if __name__ == "__main__":
    main(automation=False)  # Change to False to run only once


Scraping new jobs...
Scraping completed at 2025-03-18 13:21:07. Data saved.
